# A simple parser for LTspice files

**Scott Prahl**

May 2022

This notebook treats files with op-amps LTspice files back to a simple netlist.

In [1]:
import matplotlib.pyplot as plt
import lcapy
import ltparser

#make matplotlib hires
%config InlineBackend.figure_format='retina'

In [2]:
lt = ltparser.LTspice()
lt.read('../tests/examples/inverting-opamp.asc')
print(lt.contents)

Version 4
SHEET 1 880 680
WIRE 464 128 288 128
WIRE 624 128 544 128
WIRE -208 176 -208 144
WIRE 368 208 368 176
WIRE 80 224 -16 224
WIRE 288 224 288 128
WIRE 288 224 160 224
WIRE 336 224 288 224
WIRE 624 240 624 128
WIRE 624 240 400 240
WIRE 688 240 624 240
WIRE 336 256 176 256
WIRE -208 288 -208 256
WIRE -128 288 -208 288
WIRE -16 288 -16 224
WIRE 368 304 368 272
WIRE -208 320 -208 288
WIRE -128 320 -128 288
WIRE 176 352 176 256
WIRE -208 448 -208 400
WIRE -16 464 -16 368
WIRE 176 464 176 432
FLAG -128 320 0
FLAG -208 448 -Vs
FLAG -208 144 +Vs
FLAG 368 176 +Vs
FLAG 368 304 -Vs
FLAG 688 240 Vout
IOPIN 688 240 Out
FLAG 176 464 0
FLAG -16 464 0
SYMBOL Opamps/UniversalOpamp2 368 240 R0
SYMATTR InstName U1
SYMBOL res 176 208 R90
WINDOW 0 0 56 VBottom 2
WINDOW 3 32 56 VTop 2
SYMATTR InstName R1
SYMATTR Value 1k
SYMBOL res 560 112 R90
WINDOW 0 0 56 VBottom 2
WINDOW 3 32 56 VTop 2
SYMATTR InstName R2
SYMATTR Value 5k
SYMBOL res 192 448 R180
WINDOW 0 36 76 Left 2
WINDOW 3 36 40 Left 2
SYMATTR 

In [3]:
lt.parse()
lt.make_netlist()
lt.print_nodes()

ValueError: too many values to unpack (expected 3)

In [ ]:
lt.plot_nodes()


In [ ]:
lt = ltparser.LTspice()
lt.read('../tests/examples/audio-crossover2.asc')
lt.make_netlist()
lt.plot_nodes()
print(lt.netlist)

In [ ]:
cct=lt.circuit()
cct.draw(scale=0.5)

In [ ]:
import imageio
im = imageio.imread('../tests/examples/audio-crossover.png')
plt.figure(figsize=(14,6))
plt.imshow(im)
plt.show()

In [ ]:
print(lt.netlist)

In [ ]:
lt.make_graph()

In [ ]:
cct=lt.circuit()
cct.draw(scale=0.5)